## Imports

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import re
from tqdm import tqdm

Liste aller HTMLs

In [2]:
lst = os.listdir('pages/')

Seiten einlesen, hier zuerst mit einer Seite

In [3]:
file = open('pages/' + lst[0], 'r')
text = file.read()
h = BeautifulSoup(text, 'html.parser')

Alle Elemente, die uns interessieren

In [4]:
h.find_all('tr', {'class':'ng-scope'})

[<tr class="ng-scope" ng-repeat="row in $data"><!-- ngRepeat: col in $columns --><!-- ngIf: col.show() --><td class="ng-scope" data-title-text="business name" filter="$listCtrl.filters[col.name]" ng-if="col.show()" ng-repeat="col in $columns"><!-- ngIf: col.name === 'company' --><div class="company-name ng-scope" ng-attr-title="{{row.name}}" ng-if="col.name === 'company'" title='BÃ¤ckerei "gsund" AG'><a class="ng-binding" href="/en/search/entity/list/firm/1167884?name=b%C3%A4ckerei&amp;searchType=exact" ng-bind-html="highlight(row.name, $listCtrl.tableParams.filter(), 'name')" ng-class="{ 'strike-thru': row.status === 'GELOESCHT'}" ui-sref=".details({ ehraid: row.ehraid })">BÃ¤ckerei "gsund" AG</a></div><!-- end ngIf: col.name === 'company' --><!-- ngIf: col.name === 'legalSeat' --><!-- ngIf: col.getValue --></td><!-- end ngIf: col.show() --><!-- end ngRepeat: col in $columns --><!-- ngIf: col.show() --><td class="ng-scope" data-title-text="excerpt" filter="$listCtrl.filters[col.name]"

Alle Elemente

In [17]:
h.find_all('tr', {'class':'ng-scope'})[0].find('div')['title']

'Bäckerei zur Kettenbrücke M. Leutwiler'

In [22]:
h.find_all('tr', {'class':'ng-scope'})[0].find_next('div').find_next('div').find('a')['href']

'https://ag.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-107.083.298&lang=de'

In [35]:
h.find_all('tr', {'class':'ng-scope'})[0].find_next('div') \
    .find_next('div').find_next('div').find_next('div').find_next('div').text

'Aarau'

In [37]:
h.find_all('tr', {'class':'ng-scope'})[0].find_next('div') \
    .find_next('div').find_next('div').find_next('div') \
    .find_next('div').find_next('div').text

'AG - Aargau'

Und jetzt alles in einen For_loop packen

In [39]:
soup_lst = h.find_all('tr', {'class':'ng-scope'})

In [41]:
bäck_lst = []

for bäck in soup_lst:
    
    name = bäck.find('div')['title']
    url = bäck.find_next('div').find_next('div').find('a')['href']
    gem = bäck.find_next('div') \
        .find_next('div').find_next('div').find_next('div').find_next('div').text
    kt = bäck.find_next('div') \
        .find_next('div').find_next('div').find_next('div') \
        .find_next('div').find_next('div').text
    
    minidict = {'Bäckerei': name,
                'URL': url,
                'Gemeinde': gem,
                'Kanton': kt}
    
    bäck_lst.append(minidict)

Jetzt machen wir daraus eine Funktion

In [45]:
def get_bäcks(html_code):
    
    soup_lst = html_code.find_all('tr', {'class':'ng-scope'})
    
    bäck_lst = []
    for bäck in soup_lst:
    
        name = bäck.find('div')['title']
        url = bäck.find_next('div').find_next('div').find('a')['href']
        gem = bäck.find_next('div') \
            .find_next('div').find_next('div').find_next('div').find_next('div').text
        kt = bäck.find_next('div') \
            .find_next('div').find_next('div').find_next('div') \
            .find_next('div').find_next('div').text
    
        minidict = {'Bäckerei': name,
                'URL': url,
                'Gemeinde': gem,
                'Kanton': kt}
    
        bäck_lst.append(minidict)
    
    return bäck_lst

In [46]:
file = open('pages/' + lst[0], 'r')
text = file.read()
h = BeautifulSoup(text, 'html.parser')

In [47]:
get_bäcks(h)

[{'Bäckerei': 'Bäckerei zur Kettenbrücke M. Leutwiler',
  'URL': 'https://ag.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-107.083.298&lang=de',
  'Gemeinde': 'Aarau',
  'Kanton': 'AG - Aargau'},
 {'Bäckerei': 'Bäckerei, Konditorei Fahrni',
  'URL': 'https://be.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-107.180.240&lang=de',
  'Gemeinde': 'Dotzigen',
  'Kanton': 'BE - Bern'},
 {'Bäckerei': 'Bäckerei, Konditorei Winter, Ilic Milovan',
  'URL': 'https://zh.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-189.036.773&lang=de',
  'Gemeinde': 'Adliswil',
  'Kanton': 'ZH - Zürich'},
 {'Bäckerei': 'Bäckerei, Konditorei, Café "Zum Steinenen Trauben" Walz',
  'URL': 'https://sh.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-106.833.950&lang=de',
  'Gemeinde': 'Stein am Rhein',
  'Kanton': 'SH - Schaffhausen'},
 {'Bäckerei': 'Bäckerei, Konditorei, Confiserie Mock',
  'URL': 'https://bs.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-110.597.445&lang=de',
  'Gemeinde': 'Base

Und jetzt wenden wir das auf alle html files an

In [58]:
full_bäck_lst = []
for html_file in tqdm(lst):
    
    file = open('pages/' + html_file, 'r')
    text = file.read()
    h = BeautifulSoup(text, 'html.parser')
    
    full_bäck_lst = full_bäck_lst + get_bäcks(h)

100%|██████████| 36/36 [00:02<00:00, 12.17it/s]


Einlesen in die Liste und abspeichern

In [64]:
df = pd.DataFrame(full_bäck_lst)
df.to_csv('bäckereien_list.csv')

In [63]:
df['Kanton'].value_counts()

BE - Bern                 86
AG - Aargau               43
SG - St. Gallen           39
ZH - Zürich               35
LU - Luzern               31
GR - Graubünden           14
SO - Solothurn            13
VS - Brig (Oberwallis)    13
TG - Thurgau              12
FR - Fribourg             11
BS - Basel-Stadt          10
SZ - Schwyz                8
BL - Basel-Landschaft      7
AR - Appenzell A. Rh.      7
ZG - Zug                   5
UR - Uri                   3
GL - Glarus                3
SH - Schaffhausen          2
AI - Appenzell I. Rh.      1
NW - Nidwalden             1
Name: Kanton, dtype: int64